Requirements
==

You need to process, in Python 2, a set of client-provided trades that they will be providing in CSV format. Each trade will consist of an instrument, a price, a quantity and a timestamp - together with a number of optional columns such as the trade reference, instrument type, underlying asset, client reference. They may provide one or more files, each consisting of zero or more trades, into a common directory

Assume we need to store the trades somewhere for further processing as well provide an in-memory representation to work with them. Specifically we want to enrich the in-memory representation with the following derived information:

- For each trade we need the market value, i.e. the price multiplied by the quantity
- For each instrument we need the total market value, the closing value, and average price per day
- For each trade reference we need the constituent trades
- For each day we need to the total traded value, closing value, and the closing position

Your code should output the enriched data to the console, a file or some other store

You can submit the working code, including any relevant supporting files and specifically a README with instructions to run it, as a zip file or as a link to a GitHub repo

Please state any assumptions and describe your engineering choices




Solution
==

First of all we need to break down the whole proces into few parts:
- Generate random sample trades lists and save as `csv` file in common directory
- Read the data from all the files from the common directory into Pandas
- Store data to the database (SQlite3)
- Enrich the in-memory representation
- Output the enriched data to the file
- Display enriched data

### Importing required libraries

In [207]:
from datetime import datetime
import pandas as pd
import sqlite3 as db
import random
import time
import os

print(pd.__version__)

0.19.2


### Generating random `csv` trades lists


First of all we need to create a list of elements to appear in our dataset.
- instruments with ratios
- trade references
- instrument types
- underlying assets
- client references
- generate random timestamps

Let's create list of possible instruments from currencies array:

In [290]:
def createInstruments(currencies = {
    'USD': 1, 'EUR': 0.85, 'JPY': 110.79, 'GBP': 0.76,
    'CHF': 0.97, 'CAD': 1.24, 'AUD': 1.25, 'NZD': 1.33, 'ZAR': 13.03
    }):
    instruments = dict()
    for currency_1 in currencies.keys():
        for currency_2 in currencies.keys():
            if currency_1 != currency_2:
                ratio = currencies[currency_2] / currencies[currency_1]
                instruments[currency_1 + currency_2] = ratio
    return instruments

instruments = createInstruments()

Create references list

In [291]:
def createReferences(total):
    references = []
    for i in range(total):
        references.append('Reference_' + str(i))
    return references

totalReferences = 5

references = createReferences(totalReferences)

Create instrument types list

In [292]:
def createInstrumentTypes():
    return ['SELL', 'BUY']

instrumentTypes = createInstrumentTypes()

As the trade references, client references and underlying assets lists have the same structure, let's create one universal method to create them.

In [293]:
def createCustomList(name, total=5):
    elements = []
    for i in range(total):
        elements.append(name + '_' + str(i+1))
    return elements

tradeReferences = createCustomList('TradeReference', 5)
underlyingAssets = createCustomList('UnderlyingAsset', 10)
clientReferences = createCustomList('ClientReference', 2)

We need one more method to populate our dataset. The method to generate random timestamp when start and end dates are given.

In [294]:
def generateTimestamp(start_date, end_date):
    start_timestamp = time.mktime(time.strptime(start_date, '%b %d %Y %I:%M:%S'))
    end_timestamp = time.mktime(time.strptime(end_date, '%b %d %Y %I:%M:%S'))
    return random.randrange(start_timestamp,end_timestamp)

date = generateTimestamp('Jun 1 2010  01:33:00', 'Jun 5 2010  01:33:00')

OK so now we have all the methods to create our sample files. Let's create a method with the optional inputs of:
- output file directory path
- `csv` files count
- max trades per file
- currencies list
- trade references list
- instrument types
- underlying assets list
- client references list
- start date
- end date

In [295]:
def priceTreshhold(instrument, key):
    treshhold = 0.1 # 10%
    start = instrument[key] - instrument[key]*treshhold
    stop = instrument[key] + instrument[key]*treshhold
    return round(random.uniform(start, stop), 4)


def deleteFilesInFolder(folder):
    if os.path.isdir(folder):
        for the_file in os.listdir(folder):
            file_path = os.path.join(folder, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print(e)


def optional(value):
    return random.choice([value, ''])
            

def createSampleDataCSV(path='common_directory/',
                        filesCount=3, 
                        maxTrades=1000,
                        instruments=createInstruments(),
                        tradeReferences=createCustomList('TR', 5),
                        instrumentTypes=createInstrumentTypes(),
                        underlyingAssets=createCustomList('UA', 10),
                        clientReferences=createCustomList('CR', 3),
                        startDate='Jul 1 2017  08:30:00',
                        endDate='Jul 5 2017  01:33:00',
                        deletePrevious=True
                        ):
    if deletePrevious:
        deleteFilesInFolder(path)
        
    columns = ['Instrument','Price','Quantity',
               'Timestamp', 'Trade Reference', 
               'Instrument Type','Underlying Asset',
               'Client Reference']
    
    files = []
    for i in range(filesCount):
        fileTrades = {title : [] for title in columns }
        for k in range(random.randint(0, maxTrades)):
            currentInstrument = random.choice(instruments.keys())
            fileTrades[columns[0]].append(currentInstrument)
            fileTrades[columns[1]].append(priceTreshhold(instruments, currentInstrument))
            fileTrades[columns[2]].append(random.randint(1,10000))
            fileTrades[columns[3]].append(generateTimestamp(startDate, endDate))
            fileTrades[columns[4]].append(optional(random.choice(tradeReferences)))
            fileTrades[columns[5]].append(optional(random.choice(instrumentTypes)))
            fileTrades[columns[6]].append(optional(random.choice(underlyingAssets)))
            fileTrades[columns[7]].append(optional(random.choice(clientReferences)))
            
        trades = pd.DataFrame(fileTrades, columns=columns)
        filename = path + 'trades_list_' + str(i) + '.csv'
        trades.to_csv(filename, sep=',', encoding='utf-8', index=False)
        files.append(filename)
    return files  


Generate default sample cvs dataset files

In [296]:
createSampleDataCSV()

['common_directory/trades_list_0.csv',
 'common_directory/trades_list_1.csv',
 'common_directory/trades_list_2.csv']

### Read data from common folder files into pandas data frame

In [297]:
def readCSVFromFolder(folder, deleteFiles=True):
    tradesList = []
    for filename in os.listdir(folder):
        if filename != '.ipynb_checkpoints' and filename.endswith('.csv'):
            trades = pd.read_csv(folder + '/' + filename, sep=',')
            tradesList.append(trades.fillna(''))
    
    if deleteFiles:
        deleteFilesInFolder(folder)
    
    return pd.concat(tradesList, ignore_index=True) if len(tradesList) > 0 else pd.DataFrame()

allTrades = readCSVFromFolder(folder='common_directory', deleteFiles=False)

allTrades.tail()


,Instrument,Price,Quantity,Timestamp,Trade Reference,Instrument Type,Underlying Asset,Client Reference
2260,CADUSD,0.8355,8881,1498928415,,SELL,,CR_3
2261,EURGBP,0.9698,315,1498945147,,BUY,,CR_2
2262,NZDCAD,1.0229,3633,1499015403,TR_2,BUY,UA_10,CR_2
2263,NZDZAR,10.6843,9512,1499079813,,SELL,UA_8,
2264,EURCHF,1.1984,6135,1499088793,,BUY,,


### Store data to SQLite3 database

In [298]:
def saveDataFrameToDb(dataframe, table_name='trades', if_exists='append', db_name='database.db'):
    con = db.connect(db_name)
    normalizedToDB = dataframe.copy()
    normalizedToDB.columns = [x.lower().replace(" ", "_") for x in normalizedToDB.columns]
    normalizedToDB.to_sql(table_name, con, index=False, if_exists=if_exists)
    pass
    
saveDataFrameToDb(allTrades, if_exists='replace')

### Enriching data with derived information
1. For each trade we need the market value, i.e. the price multiplied by the quantity
2. For each instrument we need the total market value, the closing value, and average price per day
3. For each trade reference we need the constituent trades
4. For each day we need to the total traded value, closing value, and the closing position


#### 1. For each trade we need the market value
Add to pandas DataFrame additional column `Market Value` which is equal to `Price` * `Quantity`

In [299]:
def addMarketValueColumn(dataframe):
    dataframe['Market Value'] = dataframe['Price'] * dataframe['Quantity']
    return dataframe

allTrades = addMarketValueColumn(allTrades)
allTrades.head()

,Instrument,Price,Quantity,Timestamp,Trade Reference,Instrument Type,Underlying Asset,Client Reference,Market Value
0,USDZAR,13.1974,921,1499089216,TR_3,,,CR_2,12154.8054
1,JPYCHF,0.0083,7840,1498897803,,,UA_2,CR_2,65.0720
2,JPYEUR,0.0070,4631,1499140531,TR_1,,,,32.4170
3,AUDNZD,0.9623,1981,1498899990,TR_3,,,,1906.3163
4,CHFGBP,0.7721,8753,1499163518,,,UA_4,CR_2,6758.1913


#### 2. For each instrument we need the total market value, the closing value, and average price per day

Let's create a dictionary where keys are instruments and values are DataFrames containing columns `total market value`, `closing value` and `average price` listed by day

In [300]:
def getInstrumentsDailyStats(trades):
    instrumentsDailyStats = dict()
    columns = ['Day', 'Total Market Value', 'Closing Value', 'Average Price']
    for instrument in trades['Instrument'].unique():
        instrumentData = trades.loc[trades['Instrument'] == instrument]
        instrumentData = instrumentData.assign(Date = pd.Series([datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d') for timestamp in instrumentData['Timestamp']], index=instrumentData.index))
        dailyData = {title: [] for title in columns}
        
        for day in instrumentData['Date'].unique():
            currentDay = instrumentData.loc[instrumentData['Date'] == day]
            # Date
            dailyData[columns[0]].append(day)
            # Sum of Market Value column values
            dailyData[columns[1]].append(currentDay['Market Value'].sum())
            # Get price where Timestamp is max
            dailyData[columns[2]].append(currentDay.loc[currentDay['Timestamp'].idxmax()]['Price'])
            # Calculate mean of Price column
            dailyData[columns[3]].append(currentDay['Price'].mean())
            
        dailyDF = pd.DataFrame(dailyData)
        instrumentsDailyStats[instrument] = dailyDF

    return instrumentsDailyStats

instrumentsDailyStats = getInstrumentsDailyStats(allTrades)

instrument = random.choice(instrumentsDailyStats.keys())
print(instrument)
instrumentsDailyStats[instrument]

ZARNZD


,Average Price,Closing Value,Day,Total Market Value
0,0.097960,0.0950,2017-07-03,2028.5857
1,0.106900,0.1072,2017-07-01,4475.7284
2,0.103467,0.1063,2017-07-04,2168.8529
3,0.106933,0.1122,2017-07-02,1178.0938


#### 3. For each trade reference we need the constituent trades
Let's create a dictionary where keys are equal to trade references (TR) and values are DataFrames containing all constituent trades

In [301]:
def getTradeReferencesConstituents(trades):
    TrCt = dict()
    for tr in trades['Trade Reference'].unique():
        if tr != '':
            TrCt[tr] = trades.loc[trades['Trade Reference'] == tr]
    return TrCt

TRConsistTrades = getTradeReferencesConstituents(allTrades)

TR = random.choice(TRConsistTrades.keys())
print(TR)
TRConsistTrades[TR]

TR_3


,Instrument,Price,Quantity,Timestamp,Trade Reference,Instrument Type,Underlying Asset,Client Reference,Market Value
0,USDZAR,13.1974,921,1499089216,TR_3,,,CR_2,12154.8054
3,AUDNZD,0.9623,1981,1498899990,TR_3,,,,1906.3163
7,JPYUSD,0.0091,1215,1499145926,TR_3,BUY,,,11.0565
13,JPYZAR,0.1106,134,1499087083,TR_3,SELL,UA_4,CR_3,14.8204
15,AUDJPY,83.1274,9073,1499213728,TR_3,,,,754214.9002
17,USDEUR,0.8865,7309,1499083406,TR_3,BUY,,CR_3,6479.4285
25,CHFNZD,1.5005,4936,1498972509,TR_3,,,CR_2,7406.4680
31,NZDUSD,0.7932,5585,1499079648,TR_3,,UA_8,CR_2,4430.0220
35,CADCHF,0.8057,4006,1499199378,TR_3,BUY,,CR_1,3227.6342
37,ZARNZD,0.0920,49,1499059291,TR_3,SELL,,CR_3,4.5080


#### 4. For each day we need to the total traded value, closing value, and the closing position

I am not sure it I unterstand correctly this task as I am a little confused with the terminology.

- total traded value - total value of instrument trades in a stock market (sum the trades market values for each instrument by day)
- closing value - profit/loss by the end of the day (for each instrument calculate SELL Market Values minus BUY Market Values)
- closing position - the last each instrument type for the day (get the last instrument type for each instrument)

In [302]:
def getClosingPosition(currentInstrument):
    instrumentType = ''
    typeSet = currentInstrument.loc[currentInstrument['Instrument Type'] != '']
    if len(typeSet) > 0:
        instrumentType = currentInstrument.loc[typeSet['Timestamp'].idxmax()]['Instrument Type']
    return instrumentType

def getDailyStats(trades):
    dailyStats = dict()
    columns = ['Instrument', 'Total Traded Value', 'Closing Value', 'Closing Position']
    trades = trades.assign(Date = pd.Series([datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d') for timestamp in trades['Timestamp']], index=trades.index))
    for day in trades['Date'].unique():
        tradesByDay = trades.loc[trades['Date'] == day]
        dailyInstruments = {title: [] for title in columns}
        for instrument in tradesByDay['Instrument'].unique():
            currentInstrument = tradesByDay.loc[tradesByDay['Instrument'] == instrument]
            dailyInstruments[columns[0]].append(instrument)
            dailyInstruments[columns[1]].append(currentInstrument['Market Value'].sum())
            dailyInstruments[columns[2]].append(
                currentInstrument.loc[currentInstrument['Instrument Type'] == 'SELL']['Market Value'].sum() - 
                currentInstrument.loc[currentInstrument['Instrument Type'] == 'BUY']['Market Value'].sum())
            dailyInstruments[columns[3]].append(getClosingPosition(currentInstrument))
        dailyInstrumentsDF = pd.DataFrame(dailyInstruments)
        dailyStats[day] = dailyInstrumentsDF
    return dailyStats

dailyStats = getDailyStats(allTrades)

day = random.choice(dailyStats.keys())
print(day)
dailyStats[day]

2017-07-03


,Closing Position,Closing Value,Instrument,Total Traded Value
0,SELL,-1.631779e+05,USDZAR,5.716857e+05
1,BUY,-7.189704e+03,GBPCHF,8.311416e+04
2,BUY,1.173929e+06,AUDJPY,4.273229e+06
3,SELL,1.829349e+03,AUDGBP,2.515356e+04
4,SELL,-5.987765e+02,JPYZAR,3.042137e+03
5,BUY,-8.654503e+02,USDEUR,4.215068e+04
6,SELL,2.294283e+04,CHFNZD,8.181507e+04
7,SELL,-1.643232e+02,NZDUSD,2.760685e+04
8,BUY,-8.980034e+04,ZARJPY,2.427620e+05
9,BUY,2.094718e+02,ZARNZD,2.028586e+03


## Creating a function to output the data to csv files
Create folder `output` with 4 folders inside each for the different enriched data task.

In [303]:
def saveAsCsv(path, title, data, folder):
    fullPath = path + '/' + folder + '/'
    if not os.path.isdir(fullPath):
        os.makedirs(fullPath)
    data.to_csv(fullPath + title + '.csv',
                sep=',',encoding='utf-8', index=False)
    pass


def outputDataToFiles(trades, instDaily, trConsist, dailyStats, path='output', deleteBefore=True):
    if deleteBefore:
        deleteFilesInFolder(path)
    saveAsCsv(path, 'added_market_value', trades, 'Task1')
    for instrument, data in instDaily.items():
        saveAsCsv(path, instrument, data, 'Task2')
    for tr, data in trConsist.items():
        saveAsCsv(path, tr, data, 'Task3')
    for day, data in dailyStats.items():
        saveAsCsv(path, day, data, 'Task4')
        
outputDataToFiles(allTrades, instrumentsDailyStats,
                 TRConsistTrades, dailyStats)